In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from evaluation.analysis import Plotter

plt = Plotter("logs/bboxes.csv")

In [ ]:
plt._data.dtypes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def calc_bbox_edge_diff(worm_boxes: np.ndarray, mic_boxes: np.ndarray) -> np.ndarray:
    """
    Calculate the length difference between the edges of the worm's bounding boxes and the microscope's bounding boxes.

    Args:
        worm_boxes (numpy.ndarray): Array of (x, y, width, height) tuples for the worm's bounding boxes
        mic_boxes (numpy.ndarray): Array of (x, y, width, height) tuples for the microscope's bounding boxes

    Returns:
        numpy.ndarray: Array of length differences between the edges of the bounding boxes
    """
    worm_left, worm_right = worm_boxes[:, 0], worm_boxes[:, 0] + worm_boxes[:, 2]
    worm_top, worm_bottom = worm_boxes[:, 1], worm_boxes[:, 1] + worm_boxes[:, 3]
    mic_left, mic_right = mic_boxes[:, 0], mic_boxes[:, 0] + mic_boxes[:, 2]
    mic_top, mic_bottom = mic_boxes[:, 1], mic_boxes[:, 1] + mic_boxes[:, 3]

    # Calculate the length difference between the edges of the bounding boxes
    x_diff = np.maximum(0, np.maximum(worm_right - mic_right, mic_left - worm_left))
    y_diff = np.maximum(0, np.maximum(worm_bottom - mic_bottom, mic_top - worm_top))

    return np.maximum(x_diff, y_diff)


def remove_invalid_data(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove rows with negative bounding box coordinates or dimensions.
    """

    # Check if any row within each cycle has negative values in 'worm' columns
    has_negative = data.groupby('cycle')[['worm_x', 'worm_y', 'worm_w', 'worm_h']].transform('min').lt(0).any(axis=1)

    # Filter the dataframe to keep only cycles without negative values
    filtered_df = data[~has_negative].copy()

    return filtered_df

def add_speed_and_diff(data: pd.DataFrame):
    """
    Add the bounding box edge difference and worm speed to the data.
    """
    # Convert bounding box coordinates and dimensions to NumPy arrays
    worm_boxes = data[["worm_x", "worm_y", "worm_w", "worm_h"]].values
    mic_boxes = data[["mic_x", "mic_y", "mic_w", "mic_h"]].values

    # Calculate bbox edge difference
    data["bbox_edge_diff"] = calc_bbox_edge_diff(worm_boxes, mic_boxes)

    # Calculate worm speed and set it to 0 if frame values are not consecutive
    data["worm_speed"] = np.sqrt(
        (data["worm_x"].shift(-1) - data["worm_x"]) ** 2 + (data["worm_y"].shift(-1) - data["worm_y"]) ** 2
    )
    data.loc[data["frame"].diff().shift(-1) != 1, "worm_speed"] = 0

    return data


def get_cycle_stats(data: pd.DataFrame) -> pd.DataFrame:
    """
    Group the data by cycle and calculate maximum bbox edge difference and average worm speed.
    """
    return (
        data.groupby("cycle")[["bbox_edge_diff", "worm_speed"]]
        .agg({"bbox_edge_diff": "max", "worm_speed": "mean"})
        .reset_index()
    )


def plot_error_vs_speed(cycle_stats: pd.DataFrame):
    """
    Plot the maximum bounding box edge difference versus the average worm speed for each cycle.
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(cycle_stats["worm_speed"], cycle_stats["bbox_edge_diff"], s=20, alpha=0.5)
    plt.xlabel("Average Worm Speed", fontsize=14)
    plt.ylabel("Maximum Bounding Box Edge Difference", fontsize=14)
    plt.title("Maximum Bounding Box Edge Difference vs. Average Worm Speed", fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Main execution
data = pd.read_csv('logs/bboxes.csv')
data = remove_invalid_data(data)
data = add_speed_and_diff(data)
cycle_stats = get_cycle_stats(data)
plot_error_vs_speed(cycle_stats)

In [ ]:
display(cycle_stats)

In [ ]:
from evaluation.simulator import *
from evaluation.sim_controllers import *

time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.225, 0.225),
    init_position=(0, 2400),
    frame_padding_value=(255, 255, 255),
)

log_config = LogConfig(
    root_folder="logs/",
    save_mic_view=True,
    save_cam_view=True,
)


frames_path = "Archive/"
reader = FrameReader.create_from_directory(frames_path)


In [ ]:
from ultralytics import YOLO

yolo_path = "yolov8m-trained.pt"
yolo = YOLO(model=yolo_path, task="detect")

controller = LoggingController(log_config, time_config, yolo)
#controller = YoloController(time_config, yolo)
sim = Simulator(time_config, reader, controller)

In [ ]:
sim.run_simulation(visualize=True)

In [ ]:
cv.destroyAllWindows()
raise Exception("Finished")

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes